# Challenge: 

# Imports

In [94]:
import torch;
from torch import nn;
import numpy;
import matplotlib.pyplot as plt;
import pandas as pd;
from keras import layers
import sklearn;
import tensorflow as tf;
from ForwardModelNN import FeedForwardNN;

device: str = "cuda"  if torch.cuda.is_available() else "cpu";

# Installing Dependencies

In [95]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Data

## Loading the Data

In [96]:
df = pd.read_csv('dataset/baseline.csv');
column_names = df.columns;
for col in column_names: print("Feature: " + col);

df['_time'] = pd.to_datetime(df['_time'])
df

Feature: id
Feature: _time
Feature: agentID
Feature: Altitude
Feature: Latitude
Feature: Longitude
Feature: http_result
Feature: icmp_result
Feature: tcp_result
Feature: udp_result


,id,_time,agentID,Altitude,Latitude,Longitude,http_result,icmp_result,tcp_result,udp_result
0,11,2024-09-20 17:15:32+00:00,752813491,20.628773,45.495444,-73.560885,NaN,NaN,NaN,NaN
1,12,2024-09-20 17:15:30+00:00,752813491,20.628773,45.495444,-73.560885,NaN,NaN,NaN,NaN
2,13,2024-09-20 17:15:28+00:00,752813491,20.628773,45.495444,-73.560885,NaN,NaN,NaN,NaN
3,14,2024-09-20 17:15:26+00:00,752813491,20.628773,45.495444,-73.560885,NaN,NaN,NaN,NaN
4,15,2024-09-20 17:15:24+00:00,752813491,20.628773,45.495444,-73.560885,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
295,621,2024-09-20 16:53:25+00:00,752813491,23.096077,45.495682,-73.563280,NaN,NaN,NaN,NaN
296,622,2024-09-20 16:53:23+00:00,752813491,23.096077,45.495682,-73.563280,NaN,NaN,NaN,NaN
297,623,2024-09-20 16:53:21+00:00,752813491,23.339754,45.495883,-73.563161,NaN,NaN,NaN,NaN
298,624,2024-09-20 16:53:19+00:00,752813491,23.339754,45.495883,-73.563161,NaN,NaN,NaN,NaN


## Data Cleaning

We need to replace all the NaN values with 0.

In [97]:
df = df.fillna(0)
df

,id,_time,agentID,Altitude,Latitude,Longitude,http_result,icmp_result,tcp_result,udp_result
0,11,2024-09-20 17:15:32+00:00,752813491,20.628773,45.495444,-73.560885,0.0,0.0,0.0,0.0
1,12,2024-09-20 17:15:30+00:00,752813491,20.628773,45.495444,-73.560885,0.0,0.0,0.0,0.0
2,13,2024-09-20 17:15:28+00:00,752813491,20.628773,45.495444,-73.560885,0.0,0.0,0.0,0.0
3,14,2024-09-20 17:15:26+00:00,752813491,20.628773,45.495444,-73.560885,0.0,0.0,0.0,0.0
4,15,2024-09-20 17:15:24+00:00,752813491,20.628773,45.495444,-73.560885,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
295,621,2024-09-20 16:53:25+00:00,752813491,23.096077,45.495682,-73.563280,0.0,0.0,0.0,0.0
296,622,2024-09-20 16:53:23+00:00,752813491,23.096077,45.495682,-73.563280,0.0,0.0,0.0,0.0
297,623,2024-09-20 16:53:21+00:00,752813491,23.339754,45.495883,-73.563161,0.0,0.0,0.0,0.0
298,624,2024-09-20 16:53:19+00:00,752813491,23.339754,45.495883,-73.563161,0.0,0.0,0.0,0.0


In [98]:
# ensure there is no more NaN values in the table
df.isna().any()

id             False
_time          False
agentID        False
Altitude       False
Latitude       False
Longitude      False
http_result    False
icmp_result    False
tcp_result     False
udp_result     False
dtype: bool

## Splitting the Datasets 

# Machine Learning Modelling

In [105]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# From web
def create_window(target, feature, window=1, offset=0):
    feature_new, target_new = [], []
    feature_np = feature.to_numpy()
    target_np = target.to_numpy()
    
    for i in range(window, target.shape[0] - offset):
        feature_list = feature_np[i - window:i]
        feature_new.append(feature_list.reshape(window, feature_np.shape[1]))
        target_new.append(target_np[i+offset].reshape(1))
    return np.array(feature_new), np.array(target_new)


# Initialize the scaler
scaler = MinMaxScaler()
feature_columns = ['Altitude', 'Latitude', 'Longitude', 'http_result', 'icmp_result', 'tcp_result', 'udp_result']

df_scaled = df.copy()
df_scaled[feature_columns] = scaler.fit_transform(df_scaled[feature_columns])

window = 30
feature, target = create_window(df_scaled['http_result'],df_scaled[feature_columns], window=window);

x_train, x_test, y_train, y_test = train_test_split(feature, target, test_size=0.2)

# Load Model
---

In [110]:
import tensorflow as tf
from keras import layers

model = tf.keras.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=[window, 7]), # 30, 4
    layers.Dropout(0.2),

    layers.LSTM(256),
    layers.Dropout(0.2),

    layers.Dense(4)])

model.compile(
    # optimizer=tf.keras.optimizers.RMSprop(0.0001),
    optimizer='RMSProp', 
    loss='mse')

model.load_weights("./models/modelv1.h5")

2024-10-06 11:54:13.363891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-10-06 11:54:13.364592: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-10-06 11:54:13.365207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

# Perform Predictions with the Model

In [134]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import seaborn as sns

x_proto, _ = create_window(df_scaled["http_result"], df_scaled[feature_columns], window=window)
y_prob = model.predict(x_proto)

df.loc[:269, 'http_result'] = y_prob[:270, 0]  # Predictions for http_result
df.loc[:269, 'tcp_result'] = y_prob[:270, 1]   # Predictions for tcp_result
df.loc[:269, 'udp_result'] = y_prob[:270, 2]   # Predictions for udp_result
df.loc[:269, 'icmp_result'] = y_prob[:270, 3]  # Predictions for icmp_result

r2_score()

# Save the updated DataFrame to a new CSV file
df.to_csv("updated_data_with_predictions.csv", index=False)



9/9 [==============================] - 0s 2ms/step


IndexError: index 1 is out of bounds for axis 1 with size 1

# Display Metrics

In [ ]:
r2_score()

# Conclusion and Further Discussion